<a href="https://colab.research.google.com/github/lauraluebbert/kallistoXpalmdb/blob/main/kallistoXpalmDB_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# kallisto X PalmDB tutorial

This is a simple pre-release tutorial introducig the new `--aa` option of [kallisto](https://www.nature.com/articles/s41587-021-00870-2) to generate count matrices from scRNAseq data using an amino acid referece. Here, we will use this option in combination with the [PalmDB database](https://www.nature.com/articles/s41586-021-04332-2) to find viral sequences in [lung samles from APOE knock-in mice with or without COVID-19](https://www.nature.com/articles/s41586-022-05344-2).

# Install software

In [1]:
# Install ffq (https://github.com/pachterlab/ffq) and gget (https://github.com/pachterlab/gget) for simplified data download
!pip install -q gget ffq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Next, we will install the kallisto bustools wrapper [`kb-python`](https://github.com/pachterlab/kb_python) and compile kallisto from the development branch (after the official release, this will become just `pip install kb-python`):

In [2]:
# Install kb_python and kallisto from dev branch
!pip install -q git+https://github.com/pachterlab/kb_python.git@dlist
!git clone https://github.com/pachterlab/kallisto-D.git
!mkdir kallisto-D/build && cd kallisto-D/build && cmake .. && make && make install

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 22.7 MB/s eta 0:00:00
Cloning into 'kallisto-D'...
remote: Enumerating objects: 7496, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 7496 (delta 206), reused 261 (delta 181), pack-reused 7199
Receiving objects: 100% (7496/7496), 8.50 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (4958/4958), done.
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - do

# Download reference and scRNAseq data

In [3]:
# Download host transcriptome to mask host sequences in the virus index
!gget ref -d -w dna mus_musculus

Wed May 10 18:58:56 2023 INFO Fetching reference information for mus_musculus from Ensembl release: 109.
{
    "mus_musculus": {
        "genome_dna": {
            "ftp": "http://ftp.ensembl.org/pub/release-109/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz",
            "ensembl_release": 109,
            "release_date": "2022-12-13",
            "release_time": "00:07",
            "bytes": "769M"
        }
    }
}
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  769M  100  769M    0     0   480k      0  0:27:18  0:27:18 --:--:--  486k


You can find an overview of the available SRR files for the APOE knock-in study [here](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA820450&o=acc_s%3Aa).

In [4]:
# Get download links for scRNAseq data using ffq
!ffq --ftp SRR18496017

[2023-05-10 19:26:17,488]    INFO Parsing run SRR18496017
[
    {
        "accession": "SRR18496017",
        "filename": "SRR18496017_1.fastq.gz",
        "filetype": "fastq",
        "filesize": 8155785876,
        "filenumber": 1,
        "md5": "0829cf97df33da59a2d335f15ba4e673",
        "urltype": "ftp",
        "url": "ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR184/017/SRR18496017/SRR18496017_1.fastq.gz"
    },
    {
        "accession": "SRR18496017",
        "filename": "SRR18496017_2.fastq.gz",
        "filetype": "fastq",
        "filesize": 7571634748,
        "filenumber": 2,
        "md5": "f87d4358177b7748af4cafc1d51c2d1f",
        "urltype": "ftp",
        "url": "ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR184/017/SRR18496017/SRR18496017_2.fastq.gz"
    }
]


In [5]:
# Download scRNAseq data
!curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR184/017/SRR18496017/SRR18496017_1.fastq.gz
!curl -O ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR184/017/SRR18496017/SRR18496017_2.fastq.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7777M  100 7777M    0     0  22.3M      0  0:05:47  0:05:47 --:--:-- 22.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7220M  100 7220M    0     0  22.5M      0  0:05:20  0:05:20 --:--:-- 22.3M


In [6]:
# Download fasta (and annotations) containing PalmDB amino acid reference sequences
!curl -O https://raw.githubusercontent.com/lauraluebbert/kallistoXpalmdb/main/palmdb_clustered.fasta
!curl -O https://raw.githubusercontent.com/lauraluebbert/kallistoXpalmdb/main/palmdb_clustered.gtf

# Download virus ID to taxonomy mapping
!curl -O https://raw.githubusercontent.com/lauraluebbert/kallistoXpalmdb/main/palmID_to_taxonomy_mapping.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  819k  100  819k    0     0  2978k      0 --:--:-- --:--:-- --:--:-- 2978k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8051k  100 8051k    0     0  12.2M      0 --:--:-- --:--:-- --:--:-- 12.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  321k  100  321k    0     0  1459k      0 --:--:-- --:--:-- --:--:-- 1452k


# Build virus reference index from PalmDB amino acid sequences
You can find the kallisto bustools manual and tutorials [here](https://www.kallistobus.tools/).

The `--aa` argument tells kb that this is an amino acid reference.  

The `--d-list` argument is the path to the **host** transcriptome. These sequences will be masked in the index.

Since we installed kallisto from source, we have to tell kb where to find the kallisto binary using the `--kallisto` argument. This will not be neccesary anymore after the official release.

(Note: The `--d-list` and `--aa` arguments currently significantly slow down both the `kb ref` and the `kb count` commands. However, we already have some ideas to speed things up before the official release.)

In [7]:
%%time
!kb ref \
  --aa \
  --d-list Mus_musculus.GRCm39.dna.primary_assembly.fa.gz \
  -t 2 \
  -i index.idx \
  -g t2g.txt \
  -f1 transcriptome.fa \
  --kallisto /usr/local/bin/kallisto \
  palmdb_clustered.fasta \
  palmdb_clustered.gtf

[2023-05-10 19:37:41,559]   DEBUG [main] Printing verbose output
[2023-05-10 19:37:43,768]   DEBUG [main] kallisto binary located at /usr/local/bin/kallisto
[2023-05-10 19:37:43,768]   DEBUG [main] bustools binary located at /usr/local/lib/python3.10/dist-packages/kb_python/bins/linux/bustools/bustools
[2023-05-10 19:37:43,768]   DEBUG [main] Creating `tmp` directory
[2023-05-10 19:37:43,769]   DEBUG [main] Namespace(list=False, command='ref', tmp=None, keep_tmp=False, verbose=True, i='index.idx', g='t2g.txt', f1='transcriptome.fa', include_attribute=None, exclude_attribute=None, f2=None, c1=None, c2=None, d=None, k=None, t=2, d_list='Mus_musculus.GRCm39.dna.primary_assembly.fa.gz', aa=True, workflow='standard', make_unique=False, overwrite=False, kallisto='/usr/local/bin/kallisto', bustools='/usr/local/lib/python3.10/dist-packages/kb_python/bins/linux/bustools/bustools', fasta='palmdb_clustered.fasta', gtf='palmdb_clustered.gtf', feature=None, no_mismatches=False, flank=None)
[2023-05

# Align sequencing data and generate virus count matrix

In [8]:
%%time
!kb count \
  --verbose \
  --aa \
  --h5ad \
  -t 2 \
  -i index.idx \
  -g t2g.txt \
  -x split-seq \
  -o kb_results \
  --kallisto /usr/local/bin/kallisto \
  SRR18496017_1.fastq.gz \
  SRR18496017_2.fastq.gz

[2023-05-10 20:03:24,251]   DEBUG [main] Printing verbose output
[2023-05-10 20:03:26,365]   DEBUG [main] kallisto binary located at /usr/local/bin/kallisto
[2023-05-10 20:03:26,366]   DEBUG [main] bustools binary located at /usr/local/lib/python3.10/dist-packages/kb_python/bins/linux/bustools/bustools
[2023-05-10 20:03:26,366]   DEBUG [main] Creating `kb_results/tmp` directory
[2023-05-10 20:03:26,366]   DEBUG [main] Namespace(list=False, command='count', tmp=None, keep_tmp=False, verbose=True, i='index.idx', g='t2g.txt', x='split-seq', o='kb_results', w=None, r=None, t=2, m='2G', strand=None, inleaved=False, genomebam=False, aa=True, gtf=None, chromosomes=None, workflow='standard', em=False, mm=False, tcc=False, filter=None, filter_threshold=None, c1=None, c2=None, overwrite=False, dry_run=False, batch_barcodes=False, loom=False, h5ad=True, loom_names='barcode,target_name', sum='none', cellranger=False, gene_names=False, report=False, no_inspect=False, kallisto='/usr/local/bin/kallis

# Visualize virus counts 

In [9]:
# Load virus ID to taxonomy mapping
tax_df = pd.read_csv("palmID_to_taxonomy_mapping.tsv", sep='\t')
tax_df.head()

NameError: ignored

In [ ]:
# Load AnnData object retured by kb-python containing virus counts
import anndata
adata = anndata.read()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
fontsize = 14

samples = ["-30d", "-4d", "0d", "", "4h", "24h", "", "3d", "4d", "5d", "6d", "7d", "8d"]
labels = samples
virus_id = "u55137"

cellnum = []
counts = []
for sample in samples:
    count = palmdb_ebov.X[palmdb_ebov.obs.dpi_clean == sample, palmdb_ebov.var.index == virus_id].sum()
    counts.append(count)
    if count > 0:
        cellnum.append(len(nd(palmdb_ebov.X[palmdb_ebov.obs.dpi_clean == sample, palmdb_ebov.var.index == virus_id])))
    else:
        cellnum.append(0)

fontsize = 14
width = 0.4
x = np.arange(len(labels))

ax.bar(x-width/2, counts, width=width, label=f"UMI counts", color="#d42526", edgecolor="black")
ax.bar(x+width/2, cellnum, width=width, label=f"EBOV+ cells", color="#d42526", alpha=0.5, edgecolor="black")

ax.set_yscale("log")
ax.set_ylabel("Total count", fontsize=fontsize)
ax.set_xlabel("Timepoint", fontsize=fontsize)
ax.legend(loc="upper left", fontsize=fontsize)

ax.set_xticks(x, samples, rotation=0, ha="center")
# # Mark positive samples 
# for i, sample in enumerate(samples):
#     if pos[i]:
#         plt.gca().get_xticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Zaire Ebolavirus ({virus_id}) RdRP RNA molecule and cell counts", fontsize=fontsize+2)

ax.grid(True, which="both", color="lightgray")
ax.set_axisbelow(True)
    
# plt.tight_layout()

plt.savefig("figures/3.4_ebola_counts.png", dpi=300, bbox_inches="tight")
plt.savefig("figures/3.4_ebola_counts.svg", dpi=300, bbox_inches="tight")

fig.show()

Define virus filtering thresholds for bar and Krona plots:

In [ ]:
# Minimum count of virus in at least one cell
count_threshold = 2
# Minimum number of cells the virus was seen in
num_cells_threshold = 100

In [ ]:
timepoints = ['-30d', '-4d', '0d', '4h', '24h', '3d', '4d', '5d', '6d', '7d', '8d']
samples = []

# Get counts for each virus per sample
vir_fractions_df = pd.DataFrame()

for idx, virus_id in enumerate(adata.var.index):
    if adata.X[:, adata.var.index == virus_id].max() >= count_threshold and len(nd(adata.X[:, adata.var.index == virus_id].todense())) >= num_cells_threshold:
        virus_counts = []
        
        for timepoint in timepoints:
            for animal_id in adata.obs[adata.obs["dpi_clean"] == timepoint]["donor_animal"].unique():
                virus_counts.append(adata.X[(adata.obs["donor_animal"] == animal_id) & (adata.obs["dpi_clean"] == timepoint), adata.var.index == virus_id].sum())
                
                if idx == 0:
                    samples.append(timepoint + "_" + animal_id)

        utax_label = "_".join(phylogeny_data[phylogeny_data["Label"] == virus_id].values[0]).replace("_.", "")
        vir_fractions_df[utax_label] = virus_counts

vir_fractions_df["sample"] = samples

# De-fragment dataframe
vir_fractions_df = vir_fractions_df.copy()

# Set sample as index
vir_fractions_df = vir_fractions_df.set_index("sample")

# Get total virus reads per sample
vir_fractions_df["sample_total"] = vir_fractions_df.sum(axis = 1)

# Get fractions for each virus
for virus in vir_fractions_df.columns[:-1]:
    vir_fractions_df[f"{virus}_fraction"] = vir_fractions_df[virus] / vir_fractions_df["sample_total"]
    
# De-fragment dataframe
vir_fractions_df = vir_fractions_df.copy()

# Sort by maximum value accross all samples
vir_fractions_df = vir_fractions_df[vir_fractions_df.max().sort_values(ascending=False).index]

vir_fractions_df.to_csv("3.5_vir_fracs_2.csv")

vir_fractions_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
label_color="white"
fontsize = 14
width = 0.75

samples = vir_fractions_df.index.values
x = np.arange(len(samples))
bottom = np.zeros(len(samples))

# cm = plt.get_cmap('tab20')
# colormap = ["tab:red", "pink", "#9467bd", "#c4b1d4", "#2aa12b", "#98df8a", "mediumorchid", "plum", "#1f77b4", "#afc6e8", "darkorange", "#ffbc77"]
NUM_COLORS = 20
color_idx = 0
second_hatch_round = False
third_hatch_round = False
forth_hatch_round = False
hatch=""
mpl.rcParams['hatch.linewidth'] = 0.5
legend_counter = 0

for virus_column in vir_fractions_df.columns:
    if "_fraction" in virus_column:
# for virus_column in frac_cols_ebov:
        label_temp = virus_column.split("_fraction")[0]
        # Split long labels after n characters
        n = 79
        label = ("\n".join(label_temp[i:i+n] for i in range(0, len(label_temp), n))) if len(label_temp) > n else label_temp

        # Only show labels for first n viruses (else legend will be too long / break plot)
        if legend_counter <= 47:
            ax.barh(x, vir_fractions_df[virus_column].values, width, label=label, left=bottom, color=plt.cm.tab20(color_idx), hatch=hatch)
            legend_counter += 1
        else:
            ax.barh(x, vir_fractions_df[virus_column].values, width, left=bottom, color=plt.cm.tab20(color_idx), hatch=hatch)
        
        bottom += vir_fractions_df[virus_column].values

        if color_idx == NUM_COLORS-1 and not second_hatch_round and not third_hatch_round and not forth_hatch_round:
            hatch = "/"
            second_hatch_round = True
            color_idx = 0
            continue
        elif color_idx == NUM_COLORS-1 and second_hatch_round:
            hatch = "\\"
            color_idx = 0
            second_hatch_round = False
            third_hatch_round = True
            continue
        elif color_idx == NUM_COLORS-1 and third_hatch_round:
            hatch = "o"
            color_idx = 0
            third_hatch_round = False
            forth_hatch_round = True
            continue
        elif color_idx == NUM_COLORS-1 and forth_hatch_round:
            hatch = "+"
            color_idx = 0
            continue
        else:
            color_idx += 1

# Add number of total virus counts to plot and generate xtick labels
ax.set_xlim(0, 1.08)
sample_type_label = []
x_height  = 1.005
for i, sample in enumerate(samples):
    total_count = "{:,}".format(vir_fractions_df[vir_fractions_df.index == sample]["sample_total"].values[0].astype(int))  
    ax.annotate(total_count, (x_height, i), fontsize=fontsize-1, ha="left", va="center", color=label_color)
    
    sample_type_label.append(sample)

# ax.set_yscale("log")
ax.set_xlabel("Fraction of total virus reads", fontsize=fontsize, color=label_color)
ax.set_ylabel("Animal", fontsize=fontsize, color=label_color)

# leg = ax.legend(fontsize=fontsize, loc='center left', bbox_to_anchor=(1, 0.5))
leg = ax.legend(fontsize=fontsize-3, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=7)
for i, leg_txt in enumerate(leg.get_texts()):
    leg_txt.set_color("white")
leg.get_frame().set_alpha(0.2)

ax.set_yticks(x, sample_type_label, rotation=0, ha="right")
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Fraction of reads per virus species", fontsize=fontsize+2, color=label_color)
    
ax.margins(y=0.01) 

# Adjust for black background
ax.spines['bottom'].set_color('white')
ax.spines['top'].set_color('white')
ax.spines['left'].set_color('white')
ax.spines['right'].set_color('white')
ax.tick_params(axis='both', colors='white')

# plt.gca().invert_yaxis()

# plt.tight_layout()

plt.savefig("figures/3.5_palmdb_barh_per_animal.png", dpi=300, bbox_inches="tight", transparent=True)

fig.show()

# Create [Krona](https://github.com/marbl/Krona/wiki) plot 
This plot will be saved as html and can be opened in a browser.

In [ ]:
# Install Krona from source
!git clone https://github.com/marbl/Krona.git && cd Krona && ./install.pl

In [ ]:
master = pd.DataFrame()
for idx, timepoint in enumerate(palmdb_adata.obs["dpi_clean"].unique()):
    for animal_id in palmdb_adata.obs[palmdb_adata.obs["dpi_clean"] == timepoint]["donor_animal"].unique():
        adata_temp = palmdb_adata[(palmdb_adata.obs["dpi_clean"] == timepoint) & (palmdb_adata.obs["donor_animal"] == animal_id), :]

        # Add total number of counts (across all cells) for each virus ID to phylogeny data temp
        virus_ids = adata_temp.var.index.values
        total_counts = nd(adata_temp.X.sum(axis=0))
        total_count_dict = {virus_ids[i]: total_counts[i] for i in range(len(virus_ids))}

        phylogeny_data_temp = phylogeny_data.copy()
        phylogeny_data_temp['total_count'] = phylogeny_data_temp['Label'].map(total_count_dict)

        # Remove all viruses that were only detected in less than num_cells_threshold cells
        phylogeny_data_temp = phylogeny_data_temp[phylogeny_data_temp["total_count"] >= num_cells_threshold]

        # Remove non-relevent columns and change order of columns  
        phylogeny_data_temp = phylogeny_data_temp[["total_count", "phylum", "class", "order", "family", "genus", "species", "Label"]]

        # Replace dots with NaN
        phylogeny_data_temp = phylogeny_data_temp.replace(".", np.nan)

        # Add column with timepoint
        phylogeny_data_temp["timepoint"] = timepoint

        # Add column with animal id
        phylogeny_data_temp["animal_id"] = animal_id

        # Append to master dataframe
        if idx == 0:
            master = phylogeny_data_temp.copy()
        else:
            master = master.append(phylogeny_data_temp)

# Save counts + taxnomomies data to txt file
master.to_csv(f'krona.txt', sep ='\t', header=None, index=False)

# Generate Krona plot
krona_out = "figures/krona.html"
!$ktImportText krona.txt -o $krona_out -n "Viral RNAs"